# Imports and Setups

In [1]:
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 10.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 27.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 17.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 5.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [2]:
! pip install setfit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.9/45.9 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 9.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 14.1 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=898271301933d5cd6e8c83265ec8b8ac3e2b268a4e1c5228975d5916bfde02e5
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers


In [3]:
!pip install keybert

  Preparing metadata (setup.py) ... done
  Created wheel for keybert: filename=keybert-0.8.3-py3-none-any.whl size=39124 sha256=8a3d0b975b8d85f1880d676306939023dff8b454bbe6f727843b6966aaa0e1f9
  Stored in directory: /root/.cache/pip/wheels/70/88/07/1a3bc11fd1dd5f89924a02dcbca89a3015e25e8faa31f904dc
Successfully built keybert


Lendo o Arquivo

In [1]:
from google.colab import drive
drive.mount('drive')

Drive already mounted at drive; to attempt to forcibly remount, call drive.mount("drive", force_remount=True).


In [2]:
import pandas as pd
from tqdm import tqdm
tqdm.pandas()


In [3]:
num_gen = 5
perc = 5
temperatura = 0.75
tamanho = 30
numero_f = 20
i=1

In [4]:
arq = '/content/drive/My Drive/Colab Notebooks/teste_parametros_pka/perc_' +str(perc) + '/offensive/base_offensive_'+str(perc)+'_original'+str(i)+'.csv'
base_cru_1p = pd.read_csv(arq, encoding='utf-8', sep=';', decimal = ',')

In [5]:
base_cru_1p.head()

,texto,label
0,@user @user @user I’m really not on any side o...,1
1,@user @user Not all of us Conservatives are th...,0
2,Here is an Awesome Video of Alex Jones invadin...,0
3,@user Bs. He is looking for retirement money a...,0
4,@user @user @user Oh boy just wait! YOU ARE IN...,1


In [6]:
n_labels = base_cru_1p.label.nunique()
n_labels

2

In [7]:
from datasets import Dataset, DatasetDict

train = Dataset.from_pandas(base_cru_1p)

In [8]:
from sentence_transformers.losses import CosineSimilarityLoss

from setfit import SetFitModel, SetFitTrainer, sample_dataset

In [9]:
train_dataset = sample_dataset(train, label_column="label", num_samples=16)

In [10]:
train_dataset

Dataset({
    features: ['texto', 'label'],
    num_rows: 32
})

In [11]:
from sklearn.metrics import f1_score, accuracy_score
import numpy as np

def f1_multiclass(labels, preds):
      return f1_score(labels, preds, average='macro')

In [12]:
classifica = SetFitModel.from_pretrained(
    "sentence-transformers/paraphrase-mpnet-base-v2",
    # use_differentiable_head=True,
    # head_params={"out_features": 6},
)

model_head.pkl not found on HuggingFace Hub, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.


In [13]:
trainer = SetFitTrainer(
    model=classifica,
    train_dataset=train_dataset,
    # eval_dataset=eval,
    loss_class=CosineSimilarityLoss,
    # metric=f1_multiclass,
    batch_size=16,
    num_iterations=20, # The number of text pairs to generate for contrastive learning 40
    num_epochs=1, # The number of epochs to use for contrastive learning
    column_mapping={"texto": "text", "label": "label"} # Map dataset columns to text/label expected by trainer
)

In [14]:
trainer.train()

Applying column mapping to training dataset


Generating Training Pairs:   0%|          | 0/20 [00:00<?, ?it/s]

***** Running training *****
  Num examples = 1280
  Num epochs = 1
  Total optimization steps = 80
  Total train batch size = 16


Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/80 [00:00<?, ?it/s]

In [15]:
pred = classifica(base_cru_1p.texto.values)

In [16]:
f1_multiclass(base_cru_1p.label.values, pred.tolist())

0.717487922705314

In [17]:
trainer.model._save_pretrained(save_directory="./setfit_output/")

In [18]:
def retorna_word(lista_palavras):
  res = []
  for a in lista_palavras:
    res.append(a[0])
  return res

In [19]:

from keybert import KeyBERT
kw_model = KeyBERT(model = "./setfit_output/")


In [20]:
base_cru_1p['keys'] = base_cru_1p['texto'].progress_apply(lambda x:retorna_word(kw_model.extract_keywords(x, keyphrase_ngram_range=(1, 3), top_n=3, use_mmr=True, stop_words = None)))

100%|██████████| 170/170 [00:12<00:00, 13.90it/s]


In [21]:
base_cru_1p.head()

,texto,label,keys
0,@user @user @user I’m really not on any side o...,1,"[were siding with, shit jsjdhkdfhjfkfkhsjdhd m..."
1,@user @user Not all of us Conservatives are th...,0,"[gullible manny has, user user, all of us]"
2,Here is an Awesome Video of Alex Jones invadin...,0,"[vaxxed readthebook cdcwhistleblower, video of..."
3,@user Bs. He is looking for retirement money a...,0,"[looking for retirement, bs he is, new job]"
4,@user @user @user Oh boy just wait! YOU ARE IN...,1,"[user oh boy, in for surprise, that ignorant l..."


In [22]:
base_cru_1p['keys_text'] = base_cru_1p['keys'].progress_apply(lambda x: " - ".join(x))
base_cru_1p.head()

100%|██████████| 170/170 [00:00<00:00, 74359.34it/s]


,texto,label,keys,keys_text
0,@user @user @user I’m really not on any side o...,1,"[were siding with, shit jsjdhkdfhjfkfkhsjdhd m...",were siding with - shit jsjdhkdfhjfkfkhsjdhd m...
1,@user @user Not all of us Conservatives are th...,0,"[gullible manny has, user user, all of us]",gullible manny has - user user - all of us
2,Here is an Awesome Video of Alex Jones invadin...,0,"[vaxxed readthebook cdcwhistleblower, video of...",vaxxed readthebook cdcwhistleblower - video of...
3,@user Bs. He is looking for retirement money a...,0,"[looking for retirement, bs he is, new job]",looking for retirement - bs he is - new job
4,@user @user @user Oh boy just wait! YOU ARE IN...,1,"[user oh boy, in for surprise, that ignorant l...",user oh boy - in for surprise - that ignorant ...


In [23]:
pos_key = {}
for c in base_cru_1p['label'].unique():
  data = base_cru_1p[base_cru_1p['label']==c].copy()
  original = data[['keys_text', 'texto', 'label']].values.tolist()

  pos_key[c] = original

  print(len(pos_key[c]))

56
114


In [24]:
from random import sample

examples = []
generate = []
original = []
labels = []

for c in base_cru_1p['label'].unique():
  for n in range(int(2*num_gen*len(pos_key[c]))):
    if len(pos_key[c]) > (numero_f+1):
      amostra = sample(pos_key[c],(numero_f+1))
    else:
      amostra = sample(pos_key[c],len(pos_key[c]))

    gen = amostra.pop()

    examples.append(amostra)
    generate.append(gen[0])
    original.append(gen[1])
    labels.append(gen[2])


data_fim = {'examples':examples, 'generate':generate, 'original':original, 'labels':labels}

base_gen = pd.DataFrame.from_dict(data_fim)
base_gen.head()

,examples,generate,original,labels
0,"[[user good he - good he mess - he mess, @user...",he is unfit - block the abortion - cherrypicks...,@user Just the fact that #StopKavanagh REPEATE...,1
1,"[[big dick wrong - spelled big dick - user, @u...",user ohhhhhh - cried big - called me bitch,@user Ohhhhhh I cried BIG crocodile tears the ...,1
2,[[user still better - it so fucking - so fucki...,had gatlin gun - user he also - niggas that sit,@user He also said he had a gatlin gun and nig...,1
3,[[mentions lady gaga - only known immortal - r...,she is uncultured - is uncultured - user,@user she is uncultured,1
4,[[enslave the minds - must be removed - contro...,cnn he literally - user come on - for gun control,@user Come on CNN. He literally introduced hi...,1


In [25]:
base_gen.shape

(1700, 4)

In [26]:
def cria_prompt(classe, ex, synt):
  prompt = "Generate a tweet with this few keywords:"
  for el in ex:
    soma = '\nkeys: '+ el[0]+'\ntweet: '+el[1]+'\n###'
    prompt+= soma

  prompt += '\nkeys: '+ synt+'\ntweet: '
  return prompt



In [27]:
dict_label ={0:'non-offensive', 1:'offensive'}
base_gen['class_name']= base_gen['labels'].map(dict_label)
base_gen.head()

,examples,generate,original,labels,class_name
0,"[[user good he - good he mess - he mess, @user...",he is unfit - block the abortion - cherrypicks...,@user Just the fact that #StopKavanagh REPEATE...,1,offensive
1,"[[big dick wrong - spelled big dick - user, @u...",user ohhhhhh - cried big - called me bitch,@user Ohhhhhh I cried BIG crocodile tears the ...,1,offensive
2,[[user still better - it so fucking - so fucki...,had gatlin gun - user he also - niggas that sit,@user He also said he had a gatlin gun and nig...,1,offensive
3,[[mentions lady gaga - only known immortal - r...,she is uncultured - is uncultured - user,@user she is uncultured,1,offensive
4,[[enslave the minds - must be removed - contro...,cnn he literally - user come on - for gun control,@user Come on CNN. He literally introduced hi...,1,offensive


In [28]:
base_gen['prompt'] = base_gen.progress_apply(lambda x: cria_prompt(x.class_name, x.examples, x.generate), axis=1)
base_gen.head()

100%|██████████| 1700/1700 [00:00<00:00, 15338.67it/s]


,examples,generate,original,labels,class_name,prompt
0,"[[user good he - good he mess - he mess, @user...",he is unfit - block the abortion - cherrypicks...,@user Just the fact that #StopKavanagh REPEATE...,1,offensive,Generate a tweet with this few keywords:\nkeys...
1,"[[big dick wrong - spelled big dick - user, @u...",user ohhhhhh - cried big - called me bitch,@user Ohhhhhh I cried BIG crocodile tears the ...,1,offensive,Generate a tweet with this few keywords:\nkeys...
2,[[user still better - it so fucking - so fucki...,had gatlin gun - user he also - niggas that sit,@user He also said he had a gatlin gun and nig...,1,offensive,Generate a tweet with this few keywords:\nkeys...
3,[[mentions lady gaga - only known immortal - r...,she is uncultured - is uncultured - user,@user she is uncultured,1,offensive,Generate a tweet with this few keywords:\nkeys...
4,[[enslave the minds - must be removed - contro...,cnn he literally - user come on - for gun control,@user Come on CNN. He literally introduced hi...,1,offensive,Generate a tweet with this few keywords:\nkeys...


In [29]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

device = "cuda:0"

model_id = "EleutherAI/gpt-neo-2.7B"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

tokenizer = AutoTokenizer.from_pretrained(model_id)
model_4bit = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map="auto")
# model_4bit = model_4bit.to(device)

In [30]:
def gera_text(modelo, tokens, dev, ppt, temperatura, max_tokens):
  inputs = tokens(ppt, return_tensors="pt").to(dev)
  outputs = modelo.generate(**inputs, max_new_tokens=max_tokens, temperature=temperatura, do_sample =True)
  result = tokenizer.decode(outputs[0], skip_special_tokens=True)
  return result

In [31]:

print(gera_text(model_4bit, tokenizer, device,base_gen['prompt'].iloc[4], temperatura, tamanho))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Generate a tweet with this few keywords:
keys: enslave the minds - must be removed - controlled by liberals
tweet: @user @user @user @user The educational system is controlled by liberals and works to enslave the minds of our children. The chains of mental enslavement must be removed in order to save the future of our great nation 🇺🇸
###
keys: that fuck people - whore her life - in the rain
tweet: @user @user She can't tell her story while I was my car in the rain. Yes. She is a whore her life is just that. Fuck People for money. This is no exceptions
###
keys: liberals are ridiculous - are ridiculous - sexuality be an
tweet: @user Why should any sexuality be an issue for kids this age? Liberals are ridiculous
###
keys: him suck in - user yeahhhhh will - destiny cayde
tweet: @user Yeahhhhh I will admit to being a Destiny 1 Cayde Liker but God they just made him suck in 2!
###
keys: shit about you - homicide rate has - has failed to
tweet: @user @user @user And your gun control has fail

In [32]:
device = "cuda:0"
base_gen['gpt_resp'] = base_gen['prompt'].progress_apply(lambda x: gera_text(model_4bit, tokenizer, device, x, temperatura, tamanho))

100%|██████████| 1700/1700 [2:22:35<00:00,  5.03s/it]


In [33]:
base_gen.head()

,examples,generate,original,labels,class_name,prompt,gpt_resp
0,"[[user good he - good he mess - he mess, @user...",he is unfit - block the abortion - cherrypicks...,@user Just the fact that #StopKavanagh REPEATE...,1,offensive,Generate a tweet with this few keywords:\nkeys...,Generate a tweet with this few keywords:\nkeys...
1,"[[big dick wrong - spelled big dick - user, @u...",user ohhhhhh - cried big - called me bitch,@user Ohhhhhh I cried BIG crocodile tears the ...,1,offensive,Generate a tweet with this few keywords:\nkeys...,Generate a tweet with this few keywords:\nkeys...
2,[[user still better - it so fucking - so fucki...,had gatlin gun - user he also - niggas that sit,@user He also said he had a gatlin gun and nig...,1,offensive,Generate a tweet with this few keywords:\nkeys...,Generate a tweet with this few keywords:\nkeys...
3,[[mentions lady gaga - only known immortal - r...,she is uncultured - is uncultured - user,@user she is uncultured,1,offensive,Generate a tweet with this few keywords:\nkeys...,Generate a tweet with this few keywords:\nkeys...
4,[[enslave the minds - must be removed - contro...,cnn he literally - user come on - for gun control,@user Come on CNN. He literally introduced hi...,1,offensive,Generate a tweet with this few keywords:\nkeys...,Generate a tweet with this few keywords:\nkeys...


In [34]:
def separa_resp(gerar, resp):
  fim = resp.split(gerar+'\ntweet:')[1]
  return fim.split('###')[0].replace('\n', '')


In [35]:
base_gen['gen_fim'] = base_gen.progress_apply(lambda x: separa_resp(x.generate, x.gpt_resp), axis=1)
base_gen.head()

100%|██████████| 1700/1700 [00:00<00:00, 38896.95it/s]


,examples,generate,original,labels,class_name,prompt,gpt_resp,gen_fim
0,"[[user good he - good he mess - he mess, @user...",he is unfit - block the abortion - cherrypicks...,@user Just the fact that #StopKavanagh REPEATE...,1,offensive,Generate a tweet with this few keywords:\nkeys...,Generate a tweet with this few keywords:\nkeys...,@user @user @user I'm not going to call out ...
1,"[[big dick wrong - spelled big dick - user, @u...",user ohhhhhh - cried big - called me bitch,@user Ohhhhhh I cried BIG crocodile tears the ...,1,offensive,Generate a tweet with this few keywords:\nkeys...,Generate a tweet with this few keywords:\nkeys...,@user @user This is a User Ohhhhhh. He call...
2,[[user still better - it so fucking - so fucki...,had gatlin gun - user he also - niggas that sit,@user He also said he had a gatlin gun and nig...,1,offensive,Generate a tweet with this few keywords:\nkeys...,Generate a tweet with this few keywords:\nkeys...,@user he also niggas that sit @user he also ...
3,[[mentions lady gaga - only known immortal - r...,she is uncultured - is uncultured - user,@user she is uncultured,1,offensive,Generate a tweet with this few keywords:\nkeys...,Generate a tweet with this few keywords:\nkeys...,@user she is uncultured. She needs to get ...
4,[[enslave the minds - must be removed - contro...,cnn he literally - user come on - for gun control,@user Come on CNN. He literally introduced hi...,1,offensive,Generate a tweet with this few keywords:\nkeys...,Generate a tweet with this few keywords:\nkeys...,@user @user He literally said that there we...


In [36]:
teste = base_gen[base_gen['gen_fim']!= ' '].copy()
teste.shape

(1678, 8)

In [37]:
exp_lambada = []

limiar = 1.2/n_labels
base_gen = base_gen[~(base_gen['gen_fim'].isin(["ERRO",'',' ', '\n']))].copy()
preds = classifica.predict_proba(base_gen['gen_fim'].to_list()).tolist()
base_gen['preds'] = [preds[i] for i in range(len(preds))]
base_gen['prob_text'] = base_gen.progress_apply(lambda x:(x.preds[x.labels]), axis=1)
# fim = base_gen[base_gen['prob_text']>= limiar].copy()
fim = base_gen.copy()
fim.rename(columns = {'original':'texto', 'gen_fim':'generated', 'labels':'new_label'}, inplace = True)


exp_lambada.append(fim[['texto','generated', 'new_label', 'prob_text']].copy())

100%|██████████| 1678/1678 [00:00<00:00, 36991.90it/s]


In [38]:
exp_lambada[0].shape

(1678, 4)

In [39]:
base_cru_1p.shape

(170, 4)

In [40]:
exp_lambada[0].shape[0]/base_cru_1p.shape[0]

9.870588235294118

In [41]:
fim = []

for c in base_cru_1p.label.unique():
  limite = base_cru_1p[base_cru_1p.label==c].copy().shape[0]*num_gen
  gerados = exp_lambada[0][exp_lambada[0]['new_label']==c].copy()
  # gerados = gerados.drop_duplicates(subset = ['generated'])

  gerados = gerados.sort_values(by='prob_text', ascending=False)
  gerados = gerados.head(limite)
  fim.append(gerados)

In [42]:
base_aug_fim = pd.concat(fim)
base_aug_fim.head()

,texto,generated,new_label,prob_text
163,"@user @user @user Whataboutism"""" is yet anothe...",Your continual use of Left liberal Progress...,1,0.984268
557,@user Brett Kavanaugh was supposed to be a sel...,"""Not listening to Brett Kavanaugh is like...",1,0.983493
469,@user me trying to avoid the freshman boys who...,@user I had to avoid the freshman. She was ...,1,0.982764
509,@user @user She is a 🤡,She is a lying tool. She is fake a democra...,1,0.982202
279,@user He's beyond moron.,@user He's beyond stupid.,1,0.981905


In [43]:
base_aug_fim.shape[0]/base_cru_1p.shape[0]

5.0

In [44]:
arquivo = '/content/drive/My Drive/Colab Notebooks/teste_parametros_pka/perc_' +str(perc) + '/offensive/numero_f/base_offensive_numero_f_'+str(numero_f)+'.csv'
base_aug_fim.to_csv(arquivo, encoding='utf-8', index=False, sep=';', decimal = ',')